In [1]:
import re
import sys
import json


import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
from response.feature import Feature

In [3]:
F_path = "../X_y_data/response2/"
F_name = "Classify_F.pickle"
featureM = DataManager(F_path)
F = featureM.load_data(F_name)

success load : ../X_y_data/response2/Classify_F.pickle


In [5]:
model_path = "../models/response2/"
model_name = "Classify_M.pickle"
modelM = DataManager(model_path)
lr = modelM.load_data(model_name)

success load : ../models/response2/Classify_M.pickle


In [53]:

from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
clf_path = "../models/response2/"
clf_name = "detect_ignore.pickle"
clfM = DataManager(model_path)

clf = clfM.load_data(clf_name)

success load : ../models/response2/detect_ignore.pickle


In [6]:


path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [17]:
usr_sys = []
y = []
errors = ["Ignore question", "Ignore offer", "Ignore proposal", "Ignore greeting"]
for conv in convs:
    for i, ut in enumerate( conv ) :
        # if ut.is_system() and ut.is_exist_error():
        #     if conv[i-1].is_type_inclued("質問"):
        
        # システム発話と
        if ut.is_system() :
            usr_sys.append( [conv[i-1].utt, ut.utt] )
            if ut.is_error_included(errors):
                y.append(1)
            else:
                y.append(0)

In [20]:
print("len of y:{0},  counts:{1}".format(len(y), y.count(1)))

len of y:2200,  counts:354


In [21]:
from tqdm import tqdm
print(F.feature_num)
usr = []
for i, u_s in enumerate( tqdm(usr_sys) ):
    x = F.featurization(u_s[0])
    usr.append(x)
usr = np.array(usr)

  0%|          | 0/2200 [00:00<?, ?it/s]

20972


100%|██████████| 2200/2200 [07:04<00:00,  5.18it/s]


In [23]:
y_pred = lr.predict(usr)

In [24]:
fixed_y = []
# {'YN': 0, 'WH': 1, 'please': 2, 'proposal': 3, 'plain': 4}
for p in y_pred:
    if p <= 3:
        fixed_y.append(1)
    else:
        fixed_y.append(0)

In [27]:
print("len of fixed_y:{0},  counts:{1}".format(len(fixed_y), fixed_y.count(1)))

len of fixed_y:2200,  counts:680


In [25]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models
bert_path = "../../corpus/pretrained/sbert_ignore"
sbert = SentenceTransformer(bert_path)

In [26]:
def text2vec(text):
    sentences = []
    if isinstance(text, str):
        sentences = [text]
    elif isinstance(text, list):
        sentences = text
    
    return sbert.encode(sentences)

In [48]:
def text2feature(text):
    vector = text2vec(text)
    diff = np.abs( vector[0] - vector[1] )
    return np.concatenate([ vector.flatten(), diff ])

In [60]:
# 検査だ！
question_X = []
for u_s, fixed in zip(usr_sys, fixed_y):
    # 質問と仮定されたやつ
    if fixed == 1:
        # question_X.append( text2vec(u_s).flatten() )
        question_X.append( text2feature(u_s)  )

    # question_X.append( text2vec(u_s).flatten() )
    # question_X.append( text2feature(u_s) )
    

In [61]:
len(question_X)

680

In [62]:
question_pred = clf.predict(question_X)

In [63]:
# チェック
y_pred = []
fix_count = 0
for i, fixed in enumerate(fixed_y):
    # 質問
    if fixed == 1:
        # 0: not error, 1: newtral, 2: error
        if question_pred[fix_count] < 2:
            y_pred.append(0)
        else:
            y_pred.append(1)
        fix_count += 1
    # 質問ですらないならば 0
    else:
        y_pred.append(0)

In [64]:
y_pred.count(1)

408

In [65]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y, y_pred=y_pred))
print('precision = ', precision_score(y_true=y, y_pred=y_pred))
print('recall = ', recall_score(y_true=y, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y, y_pred=y_pred))

confusion matrix = 
 [[1647  199]
 [ 145  209]]
accuracy =  0.8436363636363636
precision =  0.5122549019607843
recall =  0.5903954802259888
f1 score =  0.5485564304461943


In [66]:

y_pred = [ 0 if q<2 else 1 for q in question_pred ]

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y, y_pred=y_pred))
print('precision = ', precision_score(y_true=y, y_pred=y_pred))
print('recall = ', recall_score(y_true=y, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y, y_pred=y_pred))

ValueError: Found input variables with inconsistent numbers of samples: [2200, 680]

- 前向きの検出 and S-BERT を用いた検出

        confusion matrix = 
        [[1654  192]
        [ 163  191]]
        accuracy =  0.8386363636363636
        precision =  0.49869451697127937
        recall =  0.53954802259887
        f1 score =  0.5183175033921302

- 前向きの検出 and S-BERT を用いた検出 and 特徴量変更後

        confusion matrix = 
        [[1647  199]
        [ 145  209]]
        accuracy =  0.8436363636363636
        precision =  0.5122549019607843
        recall =  0.5903954802259888
        f1 score =  0.5485564304461943

- S-BERT のみ

        confusion matrix = 
        [[1542  304]
        [  79  275]]
        accuracy =  0.8259090909090909
        precision =  0.4749568221070812
        recall =  0.7768361581920904
        f1 score =  0.5894962486602358

- SBERT のみ and 特徴量変更後

        confusion matrix = 
        [[1521  325]
        [  61  293]]
        accuracy =  0.8245454545454546
        precision =  0.4741100323624595
        recall =  0.827683615819209
        f1 score =  0.602880658436214

In [ ]:
print( y_pred.count(1), y.count(1)  )

579 354


### 所感
- Sentence BERT では未検出が少ないが，誤判定(巻き添え)も多い
    - [前向き発話以外->エラー]のパターンに対応出来ない(当たり前)
- ロジスティック回帰を掛け算すると，誤判定(巻き添え)は低減されるが未検出が多い
    - 適切に前向き発話を検出出来ていない

- 前向き発話の検出を適切に出来れば，もっと精度は上がると考えられる